In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt
from libwallerlab.projects.motiondeblur import blurkernel

# Overview
This notebook explores a SNR vs. acquisition time analysis for strobed illumination, stop and stare, and coded illumination acquisition strategies.

First, we determine a relationship between t_frame (frame rate) and t_exposure (exposure time). Then, we relate t_exposure to SNR for each method. These relationships should be smooth but non-linear.

In [2]:
# Define constants
ps = 6.5e-3 #mm
mag = 10
ps_eff_mm = ps / mag #um
n_px = np.asarray([2180, 2580])
fov = n_px * ps_eff_mm
motion_axis = 0
motion_velocity_mm_s = 25
motion_acceleration_mm_s_s = 1e4

t_settle = 0.1   #s
t_ro     = 0.01  #s

figure_directory = '/Users/zfphil/Dropbox/Berkeley/My Papers/[2018-12] MD Paper/figures/'

# Limiting Analysis

## Parameters of interest:
- Maximum SNR
- Maximum frame rate

## Camera
- full-well capacity: maximum exposure time (imaging SNR)
- readout time: maximum exposure time (imaging SNR)

## LED Array
- led update rate: Limits motion velocity
- led intensity: 

## Motion Stage
- Maximum Velocity
- Maximum Acceleration
- Settle time

## Imaging Optics
- Field of view
- Radiance collected

In [3]:
# Given frame rate

# 1. determine exposure time for strobe / coded
# 2. determine exposir

In [212]:
def calcDnfFromKernel(x):
    x = x / np.sum(x)
    if len(x) == 0:
        return np.inf
    elif np.min(np.abs(np.fft.fft(x)) ** 2) == 0:
        return np.inf
    else:
        return np.sqrt(1 / len(x) * np.sum(1 / np.abs(np.fft.fft(x)) ** 2))

def genBlurVector(kernel_length, beta=0.5, n_tests=10, metric='dnf'):
    '''
    This is a helper function for solving for a blur vector in terms of it's condition #
    '''
    kernel_list = []
    n_elements_max = math.floor(beta * kernel_length)
    for test in range(n_tests):
        indicies = np.random.randint(0, kernel_length, n_elements_max)
        kernel = np.zeros(kernel_length)
        kernel[indicies] = 1.0
        kernel_list.append(kernel)

    if metric == 'cond':
        # Determine kernel with best conditioon #
        metric_best = 1e10
        kernel_best = []
        for kernel in kernel_list:
            spectra = np.abs(np.fft.fft(kernel))
            kappa = np.max(spectra) / np.min(spectra)
            if kappa < metric_best:
                kernel_best = kernel
                metric_best = kappa
    else:
        # Determine kernel with best dnf #
        metric_best = 1e10
        kernel_best = []
        for kernel in kernel_list:
            dnf = calcDnfFromKernel(kernel)
            if dnf < metric_best:
                kernel_best = kernel
                metric_best = dnf

    return (metric_best, kernel_best)


def getOptimalDnf(kernel_size, beta=0.5, n_tests=100, metric='dnf'):
    _, x = genBlurVector(kernel_size, beta=beta, n_tests=n_tests, metric=metric)
    dnf = calcDnfFromKernel(x)
    return(dnf)

def dnf2snr(dnf, exposure_units, exposure_counts_per_unit=6553, dark_current_e=0.9, pattern_noise_e=3.9, readout_noise_e=2.5, camera_bits=16, full_well_capacity=30000):
    """
    Function which converts deconvolution noise factor to signal to noise ratio.
    Uses equations from https://www.photometrics.com/resources/learningzone/signaltonoiseratio.php and the dnf from the Agrawal and Raskar 2009 CVPR paper found here: http://ieeexplore.ieee.org/document/5206546/
    Default values are for the PCO.edge 5.5 sCMOS camera (https://www.pco.de/fileadmin/user_upload/pco-product_sheets/pco.edge_55_data_sheet.pdf)

    Args:
        dnf: Deconvolution noise factor as specified in Agrawal et. al.
        exposure_units: exposure time, time units (normally ms)
        exposure_counts_per_unit: Average number of raw image counts for a 1 unit of exposure_units exposure time
        dark_current_e: Dark current from datasheet, units electrons
        pattern_noise_e: Pattern noise from datasheet, units electrons
        readout_noise_e: Readout noise from datasheet, units electrons
        camera_bits: Number of bits in camera

    Returns:
        A 2D numpy array which indicates the support of the optical system in the frequency domain.
    """
    counts_to_e = full_well_capacity / (2 ** camera_bits - 1)
    return counts_to_e * exposure_units * exposure_counts_per_unit \
        / (dnf * math.sqrt((counts_to_e * exposure_counts_per_unit + readout_noise_e) * exposure_units + (dark_current_e + pattern_noise_e)))


def frameRateToExposure(camera_frame_rate, acquisition_strategy, 
                        motion_velocity=None, motion_velocity_max=40, motion_acceleration_max=1e3, motion_settle_time=0.25, motion_axis=1,
                        camera_bits=16, camera_exposure_counts_per_unit=150000, camera_readout_time=0.034, 
                        illumination_gamma=0.5, max_kernel_length_mm=10, illumination_min_pulse_time=4e-6, illumination_gamma_solver=0.6,
                        system_fov=(1,1), system_pixel_size=6.5e-3 / 10, debug=False, use_full_length=True):
    """
    This function fixes frame rate (frame time) and calculates exposure counts and DNFs using the following parameters:
    
    GENERAL
    - acquisition_strategy ['strobe', 'stop_and_stare' or 'coded']
    
    MOTION STAGE
    - motion_velocity [mm / s] (set to None to calculate optimally - lowest velocity to match camera_frame_rate)
    - motion_velocity_max [mm / s] (fixed)
    - motion_acceleration_max [mm / s / s] (fixed)
    - motion_settle_time [s] (fixed)
    - motion_axis [int] (fixed)
    
    CAMERA
    - camera_frame_rate [Hz] (fixed)
    - camera_bits [a.u.] (fixed)
    - camera_exposure_counts_per_unit [counts] (fixed)
    - camera_readout_time [s] (fixed)
    
    ILLUMINATION
    - illumination_gamma [in [0, 1]] (fixed)
    - illumination_min_pulse_time [s] (fixed)
    
    OPTICAL SYSTEM
    - system_fov [tuple] (fixed)
    - system_pixel_size [float] (fixed)
    """
    
    # Calculate frame time
    t_frame = 1 / camera_frame_rate
    
    # Calculate camera exposure time
    t_exp_max = t_frame - camera_readout_time
    
    # Determine maximum number of exposure units which would saturate the camera
    signal_exposure_saturate = (2 ** camera_bits - 1) / camera_exposure_counts_per_unit
    
    # Calculate velocity of not provided
    if motion_velocity is None:
        motion_velocity = min(motion_velocity_max, system_fov[motion_axis] / t_frame)
    else:
        assert system_fov[motion_axis] / t_frame > motion_velocity, "Motion velocity %g mm/s is too fast (max is %g mm/s)" % (system_fov[motion_axis] / t_frame, motion_velocity)
        
    # Calculate required LED array update speed
    t_pulse = system_pixel_size / motion_velocity
    
    # Calculate distance traveled during readout
    d_readout = motion_velocity * camera_readout_time
    
    # Calculate FOV in pixels
    fov_px = [int(_fov / system_pixel_size) for _fov in fov]

    # Ensure strobe time isn't too fast for hardware
    if t_pulse < illumination_min_pulse_time:
        print('WARNING: pulse time too short!')
        return (0,1)


    if 'stop_and_stare' in acquisition_strategy:
        # Calculate the time to start and stop
        t_start_stop = motion_velocity_max / motion_acceleration_max
        
        # Calculate the distance to start and stop
        d_start_stop = 0.5 * motion_acceleration_max * t_start_stop ** 2
        
        # Calculate movement time (constant velocity)
        t_move = (fov[motion_axis] - d_start_stop) / motion_velocity_max
        
        # Calculate exposure time (frame time - (the maximum of readout amd movement))
        signal_exposure_units = max(t_frame - max(t_move + t_start_stop, camera_readout_time), signal_exposure_saturate)
        
        # No deconvolution here
        dnf = 1
        
    else:
        # Determine pulse duration for strobe
        t_pulse = system_pixel_size / motion_velocity
        
        # Ensure pulse is not too fast
        if t_pulse < illumination_min_pulse_time:
            return (0, 1)
        
        # Strobed acquisition
        if 'strobe' in acquisition_strategy:
            # Set exposure to strobe exposure
            signal_exposure_units = t_pulse

            # No deconvolution here
            dnf = 1
            
        # Coded acquisition
        elif 'code' in acquisition_strategy:
            
            # Limit kernel_length_px to support of blur
            max_kernel_length_px = int(np.round((t_exp_max * motion_velocity) / system_pixel_size))

            # Set kernel length to be maximum length which saturates camera
            if use_full_length:
                # Set kernel length to max
                kernel_length_px = max_kernel_length_px
                
                # Determine number of pulses
                pulse_count = int(round(illumination_gamma * signal_exposure_saturate / t_pulse))
                
                # Determine illumination gamma for solver
                illumination_gamma_solver = pulse_count / kernel_length_px
            else:
                # Calculate kernel length based on illumination throughput
                kernel_length_px = int(round(signal_exposure_saturate / t_pulse / illumination_gamma))
                
                # Filter to max length
                kernel_length_px = min(kernel_length_px, max_kernel_length_px)
                
                # Use illumination gamma for solver
                illumination_gamma_solver = illumination_gamma
            
            # Ensure kernel length is nonzero
            assert kernel_length_px > 0
            
            # Determine exposure time
            signal_exposure_units = kernel_length_px * t_pulse * illumination_gamma_solver

            # Calculate DNF
            dnf = getOptimalDnf(kernel_length_px, beta=illumination_gamma_solver, n_tests=10)

    # Ensure exposure time is not negative
    if signal_exposure_units <= 0:
        signal_exposure_units = 0
    
    if debug:
        print('exposure counts: %g, max: %g' % (signal_exposure_units * camera_exposure_counts_per_unit, signal_exposure_saturate * camera_exposure_counts_per_unit))
        
    # Assume the user will always use an exposure time which does not saturate the camera
    signal_exposure_units = min(signal_exposure_units, signal_exposure_saturate)

    return (signal_exposure_units, dnf)

# Calculate camera exposure time
camera_exposure_counts_per_unit = 40000 / 7e-3

# Run some examples
frame_rate = 25
args = {'system_fov': fov, 'system_pixel_size': ps_eff_mm, 'camera_exposure_counts_per_unit': camera_exposure_counts_per_unit}

t_strobe, dnf_strobed = frameRateToExposure(frame_rate, 'strobe', **args)
snr_strobe = dnf2snr(dnf_strobed, t_strobe, exposure_counts_per_unit=camera_exposure_counts_per_unit)
print("Strobed illumination at %d fps will have photon output %g seconds and SNR %g (dnf = %g)" % (frame_rate, t_strobe, snr_strobe, dnf_strobed))

t_sns, dnf_sns = frameRateToExposure(frame_rate, 'stop_and_stare', **args)
snr_sns = dnf2snr(dnf_sns, t_sns, exposure_counts_per_unit=camera_exposure_counts_per_unit)
print("Stop-and-stare illumination at %d fps will have photon output %g seconds and SNR %g (dnf = %g)" % (frame_rate, t_sns, snr_sns, dnf_sns))

t_coded, dnf_coded = frameRateToExposure(frame_rate, 'code', illumination_gamma=0.5, illumination_gamma_solver=0.05, **args)
snr_coded = dnf2snr(dnf_coded, t_coded, exposure_counts_per_unit=camera_exposure_counts_per_unit)
print("0.50 Coded illumination at %d fps will have photon output %g seconds and SNR %g (dnf = %g)" % (frame_rate, t_coded, snr_coded, dnf_coded))

t_coded, dnf_coded = frameRateToExposure(frame_rate, 'code', illumination_gamma=0.1, **args)
snr_coded = dnf2snr(dnf_coded, t_coded, exposure_counts_per_unit=camera_exposure_counts_per_unit)
print("0.10 Coded illumination at %d fps will have photon output %g seconds and SNR %g (dnf = %g)" % (frame_rate, t_coded, snr_coded, dnf_coded))

t_coded, dnf_coded = frameRateToExposure(frame_rate, 'code', illumination_gamma=0.05, **args)
snr_coded = dnf2snr(dnf_coded, t_coded, exposure_counts_per_unit=camera_exposure_counts_per_unit)
print("0.05 Coded illumination at %d fps will have photon output %g seconds and SNR %g (dnf = %g)" % (frame_rate, t_coded, snr_coded, dnf_coded))

t_coded, dnf_coded = frameRateToExposure(frame_rate, 'code', illumination_gamma=0.01, **args)
snr_coded = dnf2snr(dnf_coded, t_coded, exposure_counts_per_unit=camera_exposure_counts_per_unit)
print("0.01 Coded illumination at %d fps will have photon output %g seconds and SNR %g (dnf = %g)" % (frame_rate, t_coded, snr_coded, dnf_coded))

Strobed illumination at 25 fps will have photon output 1.625e-05 seconds and SNR 6.18015 (dnf = 1)
Stop-and-stare illumination at 25 fps will have photon output 0.0114686 seconds and SNR 173.191 (dnf = 1)
0.50 Coded illumination at 25 fps will have photon output 0.00573625 seconds and SNR 2.75681 (dnf = 44.4265)
0.10 Coded illumination at 25 fps will have photon output 0.00115375 seconds and SNR 3.11405 (dnf = 17.6275)
0.05 Coded illumination at 25 fps will have photon output 0.00056875 seconds and SNR 2.8512 (dnf = 13.5064)
0.01 Coded illumination at 25 fps will have photon output 0.00011375 seconds and SNR 3.62981 (dnf = 4.71435)


## Plot SNR vs Frame Rate

In [213]:
frame_rates = np.arange(1,80,0.1)
snr_strobe_list = []
snr_sns_list = []
snr_coded_list_5 = []
snr_coded_list_50 = []
snr_coded_list_95 = []

camera_exposure_counts_per_unit = 200
args = {'system_fov': fov, 'system_pixel_size': ps_eff_mm, 'camera_exposure_counts_per_unit': camera_exposure_counts_per_unit}

for index, rate in enumerate(frame_rates):
    
    # Strobed illumination
    t_strobe, dnf_strobe = frameRateToExposure(rate, 'strobe', **args)
    snr_strobe_list.append(dnf2snr(dnf_strobe, t_strobe*1000, exposure_counts_per_unit=camera_exposure_counts_per_unit))

    # Stop and stare
    t_sns, dnf_sns = frameRateToExposure(rate, 'stop_and_stare', **args)
    snr_sns_list.append(dnf2snr(dnf_sns, t_sns*1000, exposure_counts_per_unit=camera_exposure_counts_per_unit))
    
    # Coded 5%
    t_coded_5, dnf_coded_5 = frameRateToExposure(rate, 'code', illumination_gamma=0.05, **args)
    snr_coded_list_5.append(dnf2snr(dnf_coded_5, t_coded_5 * 1000, exposure_counts_per_unit=camera_exposure_counts_per_unit))
    
    # Coded 50%
    t_coded_50, dnf_coded_50 = frameRateToExposure(rate, 'code', illumination_gamma=0.5, **args)
    snr_coded_list_50.append(dnf2snr(dnf_coded_50, t_coded_50 * 1000, exposure_counts_per_unit=camera_exposure_counts_per_unit))
    
    # Coded 95%
    t_coded_95, dnf_coded_95 = frameRateToExposure(rate, 'code', illumination_gamma=0.95, **args)
    snr_coded_list_95.append(dnf2snr(dnf_coded_95, t_coded_95 * 1000, exposure_counts_per_unit=camera_exposure_counts_per_unit))


# plt.style.use('classic')
plt.figure(figsize=(12,8))
plt.semilogy(frame_rates, snr_coded_list_5, 'b-', label='Coded, 5% Illuminated')
plt.semilogy(frame_rates, snr_coded_list_50, 'g-', label='Coded, 50% Illuminated')
plt.semilogy(frame_rates, snr_coded_list_95, 'y', label='Coded, 95% Illuminated')
plt.semilogy(frame_rates, snr_sns_list, 'r-', linewidth=2, label='Stop and Stare')
plt.semilogy(frame_rates, snr_strobe_list, 'w-', linewidth=2, label='Strobed')

plt.ylim((0.1, 5000))
plt.xlim((0,25))

plt.legend(fontsize=24)
plt.xlabel('Frame Rate (Hz)', fontsize=28)
plt.ylabel('SNR', fontsize=28)
ax = plt.gca()

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(24) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(24)
    
plt.grid('on', which='both')
plt.tight_layout()
# plt.savefig(figure_directory + 'strobe_sns_coded.png', transparent=True)

AssertionError: 

In [460]:
def calcDnfFromKernel(x):
    x = x / np.max(x)
    psd = np.abs(np.fft.fft(x)) ** 2
    psd /= np.max(psd)
    return np.sqrt(1 / len(x) * np.sum(1 / psd))

N = 100
gamma = 0.5
x_strobe = np.zeros(N)
x_strobe[N // 2] = 1

x_coded = np.zeros(N)
x_coded_indicies = np.random.randint(0, N, int(np.round(N * gamma)))
for ind in x_coded_indicies:
    x_coded[ind] = 1
    
print(calcDnfFromKernel(x_strobe))
print(calcDnfFromKernel(x_coded))

1.0
15.567978334879609


In [ ]:
N = 100
x = np.random.rand(N)
F = sp.linalg.dft(N)
FH = np.conj(F.T)
e_ft = np.abs(np.fft.fft(x)) ** 2 * N
AHA = FH.dot(np.diag(e_ft).dot(F))

e, _ = np.linalg.eig(AHA)

print(np.sum(np.abs(e) ** 2))
print(np.sum(np.abs(e_ft * N) ** 2))

In [413]:
x_strobe = np.zeros(N)
x_strobe[10] = 1

x_rand = np.random.rand(N)
x_rand /= sum(x_rand)
x_rand *= len(x_rand) * 0.5

# Random should have higher PSD
print(np.sum(np.abs(np.fft.fft(x_strobe)) ** 2))
print(np.sum(np.abs(np.fft.fft(x_rand)) ** 2))
print()
# Inverse random should have lower PSD
print(np.sum(1 / np.abs(np.fft.fft(x_strobe)) ** 2))
print((N // 2 - 1) / np.sum(1 / np.abs(np.fft.fft(x_rand)) ** 2))
print()
# Inverse random should have lower PSD
print(1 / np.sum(np.abs(np.fft.fft(x_strobe)) ** 2))
print(1 / np.sum(np.abs(np.fft.fft(x_rand)) ** 2))
print()

# We want the random to have a lower SNR, hwich means a higher F

1000.0
333882.7777054441

1000.0
11.26035154788475

0.001
2.9950631382437268e-06



$$ f = \sqrt{\frac{1}{m} trace(A^T A)^{-1}} $$

$$ SNR \propto \frac{1}{f} $$